## **ModelCaller**: A Python Library for Creating and Managing AI/ML Model-Calling Components 
Copyright (C) 2024, Mukesh Dalal. All rights reserved.

This notebook introduces the **ModelCaller** python library for creating and managing AI/ML model-calling components. ModelCaller facilitates calling, hosting, and registering models and functions with enhanced capabilities like automatic data sensing and caching, training, testing, and capturing supervisory and delayed feedback. It is purposefully designed for predictive and generative AI transformation and continuous improvement of enterprise software.

Contacts:
- Business inquiries: mc.business@aidaa.ai
- Press inquiries: mc.press@aidaa.ai
- Signup for email updates: mc.updates@aidaa.ai
- Feedback: mc.feedback@aidaa.ai
- [![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/modelcaller.svg?style=social&label=Follow%20%40modelcaller)](https://twitter.com/modelcaller)
- [Join ModelCaller Discord community](https://discord.gg/CgEvYuNS)

Basic imports and helper functions:

In [1]:
import random
import numpy as np
from modelcaller import ModelCaller, MCconfig, decorate_mc, wrap_mc
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO)
mc_logger = logging.getLogger('modelcaller')
mc_logger.setLevel(logging.INFO)
random.seed(42)

def generate_data(fn, count=1000, scale=100):  # generate data from a binary function that depends on a global var
    global globalx
    inputs = np.zeros((count, 3))
    outputs = np.zeros(count)
    for i in range(count):
        globalx = random.random() * scale
        x0 = random.random() * scale
        x1 = random.random() * scale
        inputs[i] = [x0, x1, globalx]
        outputs[i] = fn(x0, x1)
    return inputs, outputs

def repeat_function(fn, arity=2, count=10, scale=100): # repeatedly call a function that depends on a global var
    global globalx
    for _ in range(count):
        globalx = random.random() * scale
        args = [random.random() * scale for _ in range(arity)]
        fn(*args)

Note that we have set all log levels to INFO. 

Now let's decorate a function definition with ModelCaller (MC), while specifying all global variables used in the function as context parameters:

In [2]:
@decorate_mc(cparams=['globalx'])
def fn(x0, x1): 
    global globalx
    return 3 * x0 + x1 + globalx

Print key attributes of the MC object wrapping this function. 

In [3]:
mc = fn.mc
print('A new wrapped MC:', mc.fullstr())

A new wrapped MC: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=1, call_target:host, #functions:0, model-qualifications:[], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]


Note that host_kind=function, since the MC wraps a function. This function is called the **host** of the MC and the MC acts as a **surrogate** of this function, since all function calls will be automatically handled by the MC. Also:
- MC has no saved data: tinputs (training), toutputs, einputs (eval) and eoutputs. 
- MC has no registered function (#functions=0).
- MC has no registered models (empty model-qualifications list).
- call_target=host, i.e., calling the host does not call registered function and models.
- ncparams (number of context parameters) is correctly set to 1. 

Now let's call the host function 10 times (the default) and then print the MC again (note that feedback_fraction of calls may be randomly selected for supervisory feedback, so please be ready for that):

In [4]:
repeat_function(fn)
print('After a few function calls:', mc.fullstr())

After a few function calls: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=1, call_target:host, #functions:0, model-qualifications:[], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


Note that there are several changes in MC:
- 7 data items are saved for training and 3 for eval, consistent with edata_fraction=0.3
- the last two data items in each t* and e* list suggest that right data is being saved
- the surrogate MC indeed got called instead of the host function

Now let's register a sklearn model with this MC and then partially-print the MC (full=False):

In [5]:
from sklearn.linear_model import LinearRegression
mc.register_model(LinearRegression())
print('After fully registering the added model: ', mc.fullstr(full=False))

INFO:modelcaller: After adding a model: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=1, call_target:host, #functions:0, model-qualifications:[False], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]

INFO:modelcaller: Compare Model 0 score = 1.0 with qlty_threshold 0.95 => quality = True



After fully registering the added model:  call_target:both, #functions:0, model-qualifications:[True], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


Note that modelcaller (mc) library gave two log INFO messages, before eventually printing the MC:
- printed MC just after adding the model. Note that there is one model now that's not yet qualified (model-qualifications = [False]).
- printed evaluation score after automatically training and evaluating the added model. Since it exceeded the quality threshold, the model is now set to qualified (model-qualifications = [True]).
- the call_target was automatically updated to both, i.e., any call to the host going forward will invoke both the host as well as the registered functions and models in MC (since there is at least one of them now!). 
- we did not have to manually find training and evaluation data, manually do training and evaluation, and manually update call_target. All of this was done automatically by MC.

Since the model score is qualified, let's retire the host function completely and use just the MC as its surrogate (not both):  

In [6]:
if mc.get_call_target() == 'both': # model is qualified
    mc.update_call_target('MC') # retire the host
    print('After retiring the host: ', mc.fullstr(full=False))

After retiring the host:  call_target:MC, #functions:0, model-qualifications:[True], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


Thus, we have succesfully upgraded from a legacy SW 1.0 function to a Sw 2.0 ML model. 

Now let's register a neural sklearn model to the MC:

In [7]:
from sklearn.neural_network import MLPRegressor
midx = mc.register_model(MLPRegressor(hidden_layer_sizes=(), activation='identity'))
print('After training and evaluating the added model: ', mc.fullstr(full=False))

INFO:modelcaller: After adding a model: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=1, call_target:MC, #functions:0, model-qualifications:[True, False], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]

INFO:modelcaller: Compare Model 1 score = -5.842140673587151 with qlty_threshold 0.95 => quality = False



After training and evaluating the added model:  call_target:MC, #functions:0, model-qualifications:[True, False], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


Note that the model was again automatically trained and evaluated, but failed to qualify.

Perhaps we can add more data and retrain all the models:

In [8]:
if mc.get_call_target() == 'MC':
    xy = generate_data(mc.get_host())
    mc.add_dataset(xy[0], xy[1])
    print('After adding more data but before training: ', mc.fullstr(full=False), '\n', flush=True)
    mc.train_all_models()
    print('\nAfter training and evaluating with the new data: ', mc.fullstr(full=False))


After adding more data but before training:  call_target:MC, #functions:0, model-qualifications:[True, False], #tdata:1007, #edata:3; tinputs:...[[29.2, 8.2, 17.1], [30.8, 39.7, 84.4]]; toutputs:...[112.8, 216.6]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0] 



INFO:modelcaller: Compare Model 0 score = 1.0 with qlty_threshold 0.95 => quality = True

INFO:modelcaller: Compare Model 1 score = -5.800586906788186 with qlty_threshold 0.95 => quality = False




After training and evaluating with the new data:  call_target:MC, #functions:0, model-qualifications:[True, False], #tdata:1007, #edata:3; tinputs:...[[29.2, 8.2, 17.1], [30.8, 39.7, 84.4]]; toutputs:...[112.8, 216.6]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


Well, the neural model did not score well, even after 100 epochs (default) of training over 100 data items. Let's decrease the quality threshold and reevaluate the models:

In [9]:
if mc.isqualified(midx) == False:
    mc.qlty_threshold = -100
    mc.eval_all_models()
    print('After reevaluating all models with the new threshold: ', mc.fullstr(full=False))


INFO:modelcaller: Compare Model 0 score = 1.0 with qlty_threshold -100 => quality = True

INFO:modelcaller: Compare Model 1 score = -5.800586906788186 with qlty_threshold -100 => quality = True



After reevaluating all models with the new threshold:  call_target:MC, #functions:0, model-qualifications:[True, True], #tdata:1007, #edata:3; tinputs:...[[29.2, 8.2, 17.1], [30.8, 39.7, 84.4]]; toutputs:...[112.8, 216.6]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


Ok, both models qualified. Let's now call the host function a few more times, so that both the models are invoked:

In [10]:
if mc.isqualified(midx) == True:
    repeat_function(fn)
    print('\nAfter a few more function calls: ', mc.fullstr(full=False))


After a few more function calls:  call_target:MC, #functions:0, model-qualifications:[True, True], #tdata:1014, #edata:6; tinputs:...[[97.2, 80.8, 98.4], [43.5, 99.7, 98.8]]; toutputs:...[201.5, 144.3]; einputs:...[[47.5, 88.2, 86.7], [62.0, 12.0, 67.8]]; eoutputs:...[137.7, 119.6]


The results are very inaccurate, as expected. So, let's unregister the neural model and revert the quality threshold:

In [11]:
mc.unregister_model(1)
mc.qlty_threshold = 0.95
print('After removing the second model and reverting the threshold: ', mc.fullstr())

After removing the second model and reverting the threshold:  auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=1, call_target:MC, #functions:0, model-qualifications:[True], #tdata:1014, #edata:6; tinputs:...[[97.2, 80.8, 98.4], [43.5, 99.7, 98.8]]; toutputs:...[201.5, 144.3]; einputs:...[[47.5, 88.2, 86.7], [62.0, 12.0, 67.8]]; eoutputs:...[137.7, 119.6]


Just for fun, let's remove all training data:

In [12]:
mc.clear_dataset()
print('After removing all training data: ', mc.fullstr(full=False))

After removing all training data:  call_target:MC, #functions:0, model-qualifications:[True], #tdata:0, #edata:6; tinputs:[]; toutputs:[]; einputs:...[[47.5, 88.2, 86.7], [62.0, 12.0, 67.8]]; eoutputs:...[137.7, 119.6]


How can we get more data? Let's wrap a sensor around some external function that happens to be a copy of our original host function, so that it send data to MC:

In [13]:
@mc.wrap_sensor()
def fncopy(x0, x1, x2):  # y
    return 3 * x0 + x1 + x2

Let's repeatedly call this external function and then print the MC again:

In [14]:
repeat_function(fncopy, arity=3)
print('After a few sensor calls: ', mc.fullstr(full=False))

After a few sensor calls:  call_target:MC, #functions:0, model-qualifications:[True], #tdata:6, #edata:10; tinputs:...[[10.1, 28.6, 53.6], [33.1, 23.8, 74.6]]; toutputs:...[112.5, 197.8]; einputs:...[[2.0, 81.7, 30.3], [69.6, 9.8, 86.9]]; eoutputs:...[118.1, 305.6]


Well, 6 new samples got added to tdata and 4 to edata. This simple sensor worked!

Should we try a more complex sensor? Suppose we have an external inverse function that returns the first argument of the original host function (given the output and other arguments). Let's wrap an inverse sensor around this function, call it repeatedly, and then print MC again:

In [15]:
@mc.wrap_sensor('inverse')
def finv(y, x1, x2):  # x0
    return (y - x1 -  x2) / 3

repeat_function(finv, arity=3)
print('After a few inverse-sensor calls: ', mc.fullstr(full=False))

After a few inverse-sensor calls:  call_target:MC, #functions:0, model-qualifications:[True], #tdata:13, #edata:13; tinputs:...[[-42.7, 87.0, 58.7], [-45.8, 57.2, 96.5]]; toutputs:...[17.8, 16.3]; einputs:...[[-49.2, 83.3, 83.7], [-18.1, 74.9, 68.9]]; eoutputs:...[19.3, 89.4]


Well, 7 new samples got added to tdata and 3 to edata. This inverse sensor also worked!

Now, lets double check that computed outputs are properly saved:

In [16]:
def find_saved_output(mc, input):
    for kind in ('tdata', 'edata'):
        idx, out = mc.find_data(input, kind)
        if idx >= 0:
            return out
        
globalx = 1
y = fn(2, 3)
print(f"computed output = {y.value:.1f}")
print(f"saved output = {find_saved_output(mc, [2, 3, 1]):.1f}")

computed output = 10.0
saved output = 10.0


After this MC output (as surrogate of fn) has been computed using the model and saved, can some consumer of the output y override the value? MC provides a very convenient callback to do so:

In [17]:
y.callback(100.0)
print(f"updated output after delayed feedback = {find_saved_output(mc, [2, 3, 1])}")

updated output after delayed feedback = 100.0


This novel capability worked like a charm.

Let's try some more MC capabilities. First, let's call MC directly, instead of indirectly as the surrogate of the host function (note that we now need 3 arguments, since context arguments are included in the inputs of models):

In [18]:
repeat_function(mc, arity=3)
print('After a few MC calls: ', mc.fullstr(full=False))

After a few MC calls:  call_target:MC, #functions:0, model-qualifications:[True], #tdata:21, #edata:16; tinputs:...[[80.5, 80.2, 60.0], [65.5, 99.7, 25.9]]; toutputs:...[381.9, 322.3]; einputs:...[[46.7, 67.7, 82.4], [70.8, 57.2, 19.0]]; eoutputs:...[290.2, 288.6]


To illustrate more MC capabilities, let's create a brand new MC object in a different way, in particular without wrapping any host in it:

In [19]:
mc1 = ModelCaller(MCconfig(_ncparams=1))
print('A new unwrapped MC with one context argument: ', mc1.fullstr())

A new unwrapped MC with one context argument:  auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=None, qlty_threshold=0.95, ncparams=1, call_target:MC, #functions:0, model-qualifications:[], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]


Let's reuse the already trained model from the old mc by registering it for this new mc1, and then print it again (if you haven't edited this notebook, this call may trigger a supervisory feedback - please confirm or override the output):

In [20]:
mc1.register_model(mc.get_model(0), qualified=True) # reuse the previously qualified model
repeat_function(mc1, arity=3)
print('After a few mc calls: ', mc1.fullstr(full=False))

INFO:modelcaller: After adding a model: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=None, qlty_threshold=0.95, ncparams=1, call_target:MC, #functions:0, model-qualifications:[True], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]



After a few mc calls:  call_target:MC, #functions:0, model-qualifications:[True], #tdata:6, #edata:4; tinputs:...[[17.7, 63.4, 0.5], [76.4, 49.0, 76.4]]; toutputs:...[117.0, 354.7]; einputs:...[[74.4, 30.7, 78.8], [88.9, 50.0, 86.7]]; eoutputs:...[332.8, 403.6]


Let's demo another MC capability, i.e., ability to handle models of different types in the same MC, by registering a pytorch model to mc1 (which already has a registered sklearn model):

In [21]:
import torch
import torch.nn as nn
mc1.register_model(nn.Linear(3,1), qualified=True)
print('After adding a pytorch model: ', mc1.fullstr(full=False))
repeat_function(mc1, arity=3)
print('\nAfter a few mc calls: ', mc1.fullstr(full=False))

INFO:modelcaller: After adding a model: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=None, qlty_threshold=0.95, ncparams=1, call_target:MC, #functions:0, model-qualifications:[True, True], #tdata:6, #edata:4; tinputs:...[[17.7, 63.4, 0.5], [76.4, 49.0, 76.4]]; toutputs:...[117.0, 354.7]; einputs:...[[74.4, 30.7, 78.8], [88.9, 50.0, 86.7]]; eoutputs:...[332.8, 403.6]



After adding a pytorch model:  call_target:MC, #functions:0, model-qualifications:[True, True], #tdata:6, #edata:4; tinputs:...[[17.7, 63.4, 0.5], [76.4, 49.0, 76.4]]; toutputs:...[117.0, 354.7]; einputs:...[[74.4, 30.7, 78.8], [88.9, 50.0, 86.7]]; eoutputs:...[332.8, 403.6]

After a few mc calls:  call_target:MC, #functions:0, model-qualifications:[True, True], #tdata:13, #edata:7; tinputs:...[[66.2, 65.4, 2.0], [54.7, 47.4, 15.3]]; toutputs:...[102.0, 87.9]; einputs:...[[18.8, 94.8, 91.8], [3.7, 87.7, 25.6]]; eoutputs:...[78.3, 32.9]


Let's demo another MC capability, namely automatically adding a unique id (per MC) as a context argument to the input of each model call:

In [22]:
@decorate_mc(auto_id=True)
def f2(x0, x1):  # y
    return 3 * x0 + x1
f2(10,11)
print('A new wrapped MC with auto-id, after a function call: ', f2.mc.fullstr())

A new wrapped MC with auto-id, after a function call:  auto_cache=True, auto_id=True, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=1, call_target:host, #functions:0, model-qualifications:[], #tdata:1, #edata:0; tinputs:[[10, 11, 2308279421200]]; toutputs:[41]; einputs:[]; eoutputs:[]


Now let's demo a third way to create MC objects, in particular by wrapping a predefined function which in this happens to be a method of a fitted sklearn model:

In [23]:
m = LinearRegression()
m.fit([[1, 2, 3], [3, 4, 5]], [9, 10])
fpredict = wrap_mc(m.predict)  # wrapping a predefined function
fpredict([[10, 20, 30]])
print('A new MC, after wrapping a model.predict and calling MC: ', fpredict.mc.fullstr())

A new MC, after wrapping a model.predict and calling MC:  auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=0, call_target:host, #functions:0, model-qualifications:[], #tdata:0, #edata:1; tinputs:[]; toutputs:[]; einputs:[[10, 20, 30]]; eoutputs:[[18.0]]


Now let's demo a fourth way to create MC objects, in particular by wrapping a **model** as a host, instead of a function. We will then register the host, train the model with new data, and then calling the MC:

In [24]:
m2 = LinearRegression()
m = wrap_mc(m, kind='model', auto_id=True)
mc2 = m.mc
mc2.register_host()
mc2.train_all_models((np.array([[1, 2, 3], [3, 4, 5]], dtype=float), np.array([9, 10], dtype=float)))
m(10, 20, 30)
print('A new MC, after wrapping a model, training it, and then calling it: ', mc2.fullstr())

INFO:modelcaller: After adding a model: auto_cache=True, auto_id=True, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=model, qlty_threshold=0.95, ncparams=0, call_target:host, #functions:0, model-qualifications:[True], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]



A new MC, after wrapping a model, training it, and then calling it:  auto_cache=True, auto_id=True, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=model, qlty_threshold=0.95, ncparams=1, call_target:MC, #functions:0, model-qualifications:[True], #tdata:2, #edata:1; tinputs:[[1.0, 2.0, 3.0, 2308279452240.0], [3.0, 4.0, 5.0, 2308279452240.0]]; toutputs:[9.0, 10.0]; einputs:[[10, 20, 30, 2308279452240]]; eoutputs:[18.0]


We now demo how to wrap generative AI's foundation models, such as a large language model (LLM). This needs a HuggingFace token added as the value of the environment variable HF_TOKEN.

In [25]:
import os
import requests
HF_TOKEN = os.getenv('HF_TOKEN')
API_URL = "https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

@decorate_mc()
def llm(prompt):
    try:
        response = requests.post(API_URL, headers=headers, json=prompt)
        return response.json()[0]['generated_text']
    except Exception as e:
        logging.error(f"GPT2 raised an exception:{e} for the prompt:{prompt}")
        return(e)

llm("I want to")
llm("I do not want to")
print('A new MC, after two calls to GPT2:', llm.mc.fullstr())

A new MC, after two calls to GPT2: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=0, call_target:host, #functions:0, model-qualifications:[], #tdata:0, #edata:2; tinputs:[]; toutputs:[]; einputs:[['I want to'], ['I do not want to']]; eoutputs:[I want to be your friend,"

He was dead.

I wanted this feeling to wash over them forever. It felt like a piece of paper.

My sister could see. She was afraid of the cold the other day and, I do not want to see that the US does not want to have a similar environment," said Alisa Shradyim, head of the International Centre for Palestinian Rights Research, which is working with the US State Department on making UNRWA a]


Since a MC object provides the core API of a model (call, train and eval), it can be also wrapped as a host or registered as a model in another MC:

In [26]:
llm = wrap_mc(llm)
llm("Shakespeare wrote")
print('A new MC that nests previous MC, after one call to GPT2:', llm.mc.fullstr())

A new MC that nests previous MC, after one call to GPT2: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=0, call_target:host, #functions:0, model-qualifications:[], #tdata:0, #edata:1; tinputs:[]; toutputs:[]; einputs:[['Shakespeare wrote']]; eoutputs:[Shakespeare wrote, The Act of Oration: "I shall not sing, neither will I sing. I shall not speak, as he who takes his seat at the right hand of the most holy prince, who took up the office of the supreme]


Let's test whether the nested MC also saved the data from the new call:

In [27]:
nested_llm = llm.mc.get_host()
print('The nested MC:', nested_llm.mc.fullstr())

The nested MC: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, host_kind=function, qlty_threshold=0.95, ncparams=0, call_target:host, #functions:0, model-qualifications:[], #tdata:0, #edata:3; tinputs:[]; toutputs:[]; einputs:...[['I do not want to'], ['Shakespeare wrote']]; eoutputs:...[I do not want to see that the US does not want to have a similar environment," said Alisa Shradyim, head of the International Centre for Palestinian Rights Research, which is working with the US State Department on making UNRWA a, Shakespeare wrote, The Act of Oration: "I shall not sing, neither will I sing. I shall not speak, as he who takes his seat at the right hand of the most holy prince, who took up the office of the supreme]
